In this sript, the Isomap embedding matrix in taken from the pandas dataFrame
to perform an analysis on the distribution the components under specific groups, 
them being motor score groups of data or high-risk groups

In [1]:
## Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC
from sklearn import metrics
from scipy.stats import kruskal, mannwhitneyu
import statsmodels.stats.multitest as smm
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_curve, auc

In [98]:
dataset_unit = pd.read_csv("./output/dataset_unit.csv")
# no outliers (inclides all criteria)
dataset_unit_nds = dataset_unit[dataset_unit.outlier_diff_shape != 1]
dataset_unit_nds = dataset_unit_nds[dataset_unit_nds.vis_QA != 1.0]

with open(f"/mrhome/marlenesga/Documents/output/dim10_k50_eucl_23_new_reconst/isomap_matrix_dim10_k50_eucl_23_new_reconst.npy", 'rb') as f:
    isomap_mtx = np.load(f)
    
#####################################
# ADD ISOMAP DATA
#####################################
for comp in range(isomap_mtx.shape[1]): #10 components
    dataset_unit_nds[f"isomap_comp_{comp+1}"] = isomap_mtx[:,comp]
dataset_unit_nds

,famlbnr,hemisphere,HighRiskStatus_v11,MRI_age_v11,Mvmt_age_child_v11,Mvmt_bal_stsc_v11,Mvmt_bf_stscr_v11,Mvmt_hand_v11,Mvmt_hm_stscr_v11,Sex_child_v11,...,isomap_comp_12,isomap_comp_13,isomap_comp_14,isomap_comp_15,isomap_comp_16,isomap_comp_17,isomap_comp_18,isomap_comp_19,outlier_diff_shape,outlier_std
0,3,left,K,12.188912,11.586585,10.0,13.0,2.0,12.0,0.0,...,0.035721,-0.452041,0.880670,-0.722755,-0.247877,1.128720,0.167213,-0.350594,0,0
1,3,right,K,12.188912,11.586585,10.0,13.0,2.0,12.0,0.0,...,0.056202,-0.357871,0.195984,-2.438467,0.834821,0.511466,0.729010,-1.541064,0,0
6,10,left,SZ,11.718001,11.583847,4.0,10.0,1.0,1.0,1.0,...,0.084010,1.138845,-2.286611,-0.483698,-1.795066,1.078357,1.160150,0.886613,0,0
7,10,right,SZ,11.718001,11.583847,4.0,10.0,1.0,1.0,1.0,...,0.160852,-1.573918,-1.861005,-0.007328,-2.401612,0.884988,-0.620183,-1.827109,0,0
8,13,left,K,12.287474,12.249144,14.0,15.0,2.0,11.0,0.0,...,-0.219500,-0.457744,-0.461576,-0.059101,-1.557460,-0.295891,-0.577185,-0.367239,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,518,right,K,12.353183,12.112252,10.0,9.0,2.0,8.0,1.0,...,-0.706846,-0.817725,-0.729056,0.870509,-0.183426,-0.970751,-0.322440,-0.136960,0,0
600,519,left,K,12.533881,12.528405,2.0,10.0,2.0,9.0,1.0,...,0.680085,-0.147762,-0.390602,-0.978639,1.944333,-0.220556,-0.141750,-1.342559,0,0
601,519,right,K,12.533881,12.528405,2.0,10.0,2.0,9.0,1.0,...,-0.352158,-0.820471,-0.320339,-1.227458,-0.618391,-1.909015,0.961783,-0.696881,0,0
602,521,left,SZ,11.118412,11.091034,6.0,7.0,1.0,6.0,1.0,...,-1.704726,0.057766,-1.083894,-0.102393,0.267967,-0.327549,0.252124,-1.348282,0,0


In [455]:
# PREDICTION OF MOTOR SCORES BASED ON ISOMAP COMPONENTS

data = dataset_unit_nds

dichotomiz_data = data.copy()
dichotomiz_data = dichotomiz_data.reset_index(drop=True)

for i, val in enumerate(data['Mvmt_bf_stscr_v11']):
    if val <= 5:
        dichotomiz_data.at[i, 'Mvmt_bf_stscr_v11_bin'] = 0
    elif val >= 13:
        dichotomiz_data.at[i, 'Mvmt_bf_stscr_v11_bin'] = 1
    else:
        dichotomiz_data = dichotomiz_data.drop(i)

dichotomiz_data = dichotomiz_data.reset_index(drop=True)

print(dichotomiz_data['Mvmt_bf_stscr_v11_bin'].value_counts())


x = np.column_stack((dichotomiz_data['isomap_comp_1'], dichotomiz_data['isomap_comp_2'], dichotomiz_data['isomap_comp_3'], \
                     dichotomiz_data['isomap_comp_4'],dichotomiz_data['isomap_comp_5'],dichotomiz_data['isomap_comp_6'],dichotomiz_data['isomap_comp_7'],\
                     dichotomiz_data['isomap_comp_8'], dichotomiz_data['isomap_comp_9'], dichotomiz_data['isomap_comp_10']))

# Ensure target variable is of type int
dichotomiz_data['Mvmt_bf_stscr_v11_bin'] = dichotomiz_data['Mvmt_bf_stscr_v11_bin'].astype(int)


y = dichotomiz_data['Mvmt_bf_stscr_v11_bin']
y = np.array(y)

# Values of C to try
C_values = np.logspace(-5, 5, 10, base=2)
gamma_values = np.logspace(-5, 5, 10, base=2)
param_grid = {'C': C_values, 'gamma': gamma_values}

# Create StratifiedKFold object
skf = StratifiedKFold(n_splits=10, shuffle=True) #andom_state=42)

# Support Vector Classifier
svc = SVC(kernel='linear', class_weight="balanced")

# GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(svc, param_grid, cv=skf, scoring='accuracy')

# Fit GridSearchCV
grid_search.fit(x, y)

# Best parameters
best_params = grid_search.best_params_
best_C = best_params['C']
best_gamma = best_params['gamma']
print(f"Best C value: {best_C}")
print(f"Best gamma value: {best_gamma}")

# DataFrame to store results
results_df = pd.DataFrame(columns=['Fold', 'Training Accuracy', 'Test Accuracy', 'Test F1-Score'])

# Evaluate the model using the best parameters on the same cross-validation
k = 1
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

plt.figure(figsize=(10, 8))

coef_dict = np.zeros((10,10))

for train_index, test_index in skf.split(x, y):
    print('fold', k)
    x_train_fold, x_test_fold = x[train_index], x[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Train with the best parameter
    clf = SVC(kernel='linear', C=best_C, gamma=best_gamma, class_weight="balanced")
    clf.fit(x_train_fold, y_train_fold)


    # Predictions
    y_pred_train = clf.predict(x_train_fold)
    y_pred_test = clf.predict(x_test_fold)
    y_score_test = clf.decision_function(x_test_fold)

    # Metrics
    acc_train = metrics.accuracy_score(y_train_fold, y_pred_train)
    acc_test = metrics.accuracy_score(y_test_fold, y_pred_test)
    f1_test = metrics.f1_score(y_test_fold, y_pred_test, average='weighted')
    fpr, tpr, _ = roc_curve(y_test_fold, y_score_test)
    roc_auc = auc(fpr, tpr)
    
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    aucs.append(roc_auc)

    print('train_acc', f"{acc_train:.2f}", 'test acc', f"{acc_test:.2f}", 'test F1', f"{f1_test:.2f}", 'test AUC', f"{roc_auc:.2f}")

    # Store results in DataFrame
    results_df.at[k, 'Fold'] = k
    results_df.at[k, 'Training Accuracy'] = acc_train
    results_df.at[k, 'Test Accuracy'] = acc_test
    results_df.at[k, 'Test F1-Score'] = f1_test
    results_df.at[k, 'Test AUC'] = roc_auc

    # Extract coefficients
    coef_dict[k-1,:] = clf.coef_

    k += 1

# Print results table
print("\nCross-Validation Results:")
print(results_df)

# Plot mean ROC curve
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

# Plotting random chance line
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', alpha=.8)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

Mvmt_bf_stscr_v11_bin
1    52
0    52
Name: count, dtype: int64
Best C value: 3.1748021039363996
Best gamma value: 0.03125
fold 1
train_acc 0.69 test acc 0.64 test F1 0.62 test AUC 0.70
fold 2
train_acc 0.69 test acc 0.55 test F1 0.54 test AUC 0.67
fold 3
train_acc 0.72 test acc 0.55 test F1 0.55 test AUC 0.60
fold 4
train_acc 0.70 test acc 0.45 test F1 0.45 test AUC 0.37
fold 5
train_acc 0.69 test acc 0.60 test F1 0.58 test AUC 0.64
fold 6
train_acc 0.68 test acc 0.30 test F1 0.23 test AUC 0.44
fold 7
train_acc 0.66 test acc 0.80 test F1 0.80 test AUC 0.68
fold 8
train_acc 0.73 test acc 0.50 test F1 0.49 test AUC 0.60
fold 9
train_acc 0.67 test acc 0.70 test F1 0.67 test AUC 0.88
fold 10
train_acc 0.68 test acc 0.50 test F1 0.49 test AUC 0.60

Cross-Validation Results:
   Fold Training Accuracy Test Accuracy Test F1-Score  Test AUC
1     1          0.688172      0.636364      0.616883  0.700000
2     2          0.688172      0.545455      0.537685  0.666667
3     3           0.72043  

In [ ]:
# VISUALIZE COMPONENT DISTRIBUTIONS FOR SPECIFIC GROUPS
%matplotlib tk

df_hem = dataset_unit_nds.groupby('hemisphere')
df_side = df_hem.get_group('left')

plt.figure()
ax = sns.boxplot(data=df_side, x="isomap_comp_8", hue="HighRiskStatus_v11", palette='viridis', gap=.1)
plt.xlabel('component 8')
plt.ylabel('Familial High-Risk')
sns.move_legend(ax, "upper right", bbox_to_anchor=(1.15, 1))

In [ ]:
# TEST DIFFERENCES IN PAIRS OF DISTRIBUTIONS IN SPEFIFIC GROUPS

%matplotlib tk

df_hem = dataset_unit_nds.groupby('hemisphere')
# set hemisphere
df_side = df_hem.get_group('left')

data = df_side

dichotomiz_data = data.copy()
dichotomiz_data = dichotomiz_data.reset_index(drop=True)

for i, val in enumerate(data['Mvmt_hm_stscr_v11']):
    if val <= 5:
        dichotomiz_data.at[i, 'Mvmt_hm_stscr_v11_bin'] = 0
    elif val >= 13:
        dichotomiz_data.at[i, 'Mvmt_hm_stscr_v11_bin'] = 1
    else:
        dichotomiz_data = dichotomiz_data.drop(i)

dichotomiz_data = dichotomiz_data.reset_index(drop=True)

plt.figure()
ax = sns.boxplot(data=dichotomiz_data, x="isomap_comp_2", hue="Mvmt_hm_stscr_v11_bin", palette='viridis', gap=.1)
plt.xlabel('component 1')
plt.ylabel('Motor Scores')
handles, _ = plt.gca().get_legend_handles_labels()
plt.legend(handles=handles, labels=["<=5th Percentile",">=13th Percentile"], loc='upper left', bbox_to_anchor=(0.9, 1))
#sns.move_legend(ax, "upper right", bbox_to_anchor=(1.15, 1))

# Performing the Wilcoxon rank-sum test / Mann–Whitney 𝑈
group_0 = dichotomiz_data[dichotomiz_data['Mvmt_hm_stscr_v11_bin'] == 0]['isomap_comp_2']
group_1 = dichotomiz_data[dichotomiz_data['Mvmt_hm_stscr_v11_bin'] == 1]['isomap_comp_2']

stat, p_value = mannwhitneyu(group_0, group_1, alternative='two-sided')

print(f'Wilcoxon rank-sum test statistic: {stat}')
print(f'p-value: {p_value}')

#If the p-value is less than a chosen significance level (e.g., 0.05), you can reject the null 
# hypothesis that the distributions of the two groups are the same.

In [ ]:
# FAMILIAL HIGH RISK RELATED COMPONENT DISTRIBUTION DIFFERENCES TESTING (THREE GROUPS)

df_hem = dataset_unit_nds.groupby('hemisphere')

# set hemisphere
df_side = df_hem.get_group('left')

data = df_side

# Example data: Isomap coordinates and group labels
np.random.seed(0)
isomap_coords = np.column_stack((data['isomap_comp_1'], data['isomap_comp_2'],data['isomap_comp_3'], \
                     data['isomap_comp_4'],data['isomap_comp_5'],data['isomap_comp_6'],data['isomap_comp_7'],\
                        data['isomap_comp_8'], data['isomap_comp_9'], data['isomap_comp_10']))

#np.random.rand(60, 5)  # 60 samples, 5 Isomap components
groups = np.array(data['HighRiskStatus_v11'])
#np.repeat(['Group1', 'Group2', 'Group3'], 20)  # 3 groups, 20 samples each

# Create a DataFrame
df = pd.DataFrame(isomap_coords, columns=[f'Component_{i+1}' for i in range(isomap_coords.shape[1])])
df['Group'] = groups


# Perform Kruskal-Wallis test for each Isomap component
results = {}
for component in df.columns[:-1]:  # Exclude the 'Group' column
    print(f'Analyzing {component}...')

    group_data = [df[df['Group'] == group][component] for group in df['Group'].unique()]
    statistic, p_value = kruskal(*group_data)
    results[component] = p_value  # Store the p-value

    print(f'Kruskal-Wallis test for {component}: statistic={statistic}, p-value={p_value}')
    print()

# Identify components significantly related to the groups
significant_components = [component for component, p_value in results.items() if p_value < 0.05]
print(f'Significant components: {significant_components}')

# Perform post-hoc analysis if there are significant components
if significant_components:
    for component in significant_components:
        print(f'Post-hoc analysis for {component}...')
        comparisons = []
        groups = df['Group'].unique()
        for i in range(len(groups)):
            for j in range(i+1, len(groups)):
                g1 = groups[i]
                g2 = groups[j]
                stat, p = mannwhitneyu(df[df["Group"] == g1][component], df[df["Group"] == g2][component])
                comparisons.append((g1, g2, p))

        # Adjust p-values for multiple comparisons using Bonferroni correction
        p_values = [comp[2] for comp in comparisons]
        reject, pvals_corrected, _, _ = smm.multipletests(p_values, alpha=0.05, method='bonferroni')

        # Print the results
        for idx, comp in enumerate(comparisons):
            g1, g2, p = comp
            print(f"Comparison {g1} vs {g2}: p-value={pvals_corrected[idx]} {'(significant)' if reject[idx] else ''}")

# Visualize significant components
for component in significant_components:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='Group', y=component, palette='viridis')
    plt.title(f'{component} by Group')
    plt.xlabel('Group')
    plt.ylabel(component)
    plt.show()


Analyzing Component_1...
Kruskal-Wallis test for Component_1: statistic=2.2331558801045617, p-value=0.3273982562698934

Analyzing Component_2...
Kruskal-Wallis test for Component_2: statistic=3.4198742250027863, p-value=0.18087716717205826

Analyzing Component_3...
Kruskal-Wallis test for Component_3: statistic=0.30125065122905426, p-value=0.860169921927934

Analyzing Component_4...
Kruskal-Wallis test for Component_4: statistic=0.0914897577258671, p-value=0.9552856446650082

Analyzing Component_5...
Kruskal-Wallis test for Component_5: statistic=0.7567759873001592, p-value=0.6849646871530832

Analyzing Component_6...
Kruskal-Wallis test for Component_6: statistic=3.732115170733266, p-value=0.15473248051413993

Analyzing Component_7...
Kruskal-Wallis test for Component_7: statistic=0.41304255167136716, p-value=0.8134089553175835

Analyzing Component_8...
Kruskal-Wallis test for Component_8: statistic=7.064869013636667, p-value=0.029233659621652967

Analyzing Component_9...
Kruskal-Wall

/tmp/ipykernel_2547521/970073477.py:72: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, x='Group', y=component, palette='viridis')
/tmp/ipykernel_2547521/970073477.py:76: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
